In [1]:
%load_ext autoreload  
%autoreload 2 
%matplotlib inline

In [2]:
import torch 
from tqdm import tqdm
import json
from datasets import load_dataset
from sentence_transformers import CrossEncoder

In [3]:
# get instances
dataset = load_dataset('e2e_nlg')


Using custom data configuration default
Reusing dataset e2e_nlg (/Users/garylai/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430)
100%|██████████| 3/3 [00:00<00:00, 172.53it/s]


In [4]:
dataset = dataset['train']; len(dataset)

42061

In [5]:
import random

# Write to JSON
unique_sentences = set()

Instances = []
for i in range(3000):
    # prevent duplicates
    if dataset[i]['human_reference'] in unique_sentences or dataset[i]['meaning_representation'] in unique_sentences:
        continue
    
    unique_sentences.add(dataset[i]['human_reference'])
    unique_sentences.add(dataset[i]['meaning_representation'])
    
    Instances.append({
        # "input": dataset[i]['meaning_representation'],
        # "output": [dataset[i]['human_reference']]
        "input": dataset[i]['human_reference'],
        "output": [dataset[i]['meaning_representation']]
    })

In [9]:
len(Instances)

2162

In [11]:
random.shuffle(Instances)

first instance:  {'input': 'name[The Mill], eatType[restaurant], area[riverside], near[The Rice Boat]', 'output': ['A restaurant called The Mill, can be found near the riverside next to The Rice Boat.']}


In [13]:
for i, instance in enumerate(Instances):
    print(instance)
    if i > 20:
        break

{'input': 'name[The Mill], eatType[restaurant], area[riverside], near[The Rice Boat]', 'output': ['A restaurant called The Mill, can be found near the riverside next to The Rice Boat.']}
{'input': 'name[Midsummer House], food[Indian], customer rating[low], near[Café Rouge]', 'output': ['Located near Café Rouge Midsummer House is an Indian restaurant with a poor customer rating.']}
{'input': 'name[Taste of Cambridge], eatType[restaurant], priceRange[cheap], customer rating[average]', 'output': ['Taste of Cambridge is an average rated and cheap restaurant option.']}
{'input': 'name[The Eagle], eatType[coffee shop], food[Indian], priceRange[more than £30], customer rating[high], area[city centre], familyFriendly[no], near[Burger King]', 'output': ['In the city centre, near Burger King, you will find The Eagle. It is a non children-friendly coffee shop that serves Indian food. While its price range is quite high, more than £30, it has a high customer ratings.']}
{'input': 'name[Browns Camb

In [14]:
task_json = {
    "Contributors": [
        "Gary Haizhi Lai"
    ],
    "Source": [
        "e2e"
    ],
    "Categories": [
        "Text Generation"
    ],
    "Definition": "In this task, we ask you convert a data table into fluent natural-sounding English sentences. The input is a string of key-value pairs; the output is a natural and grammatical English sentence containing all the information from the input string",
    "Positive Examples": [
        {
            "input": "name[Aromi], eatType[restaurant], food[English], area[city centre]",
            "output": "Aromi is an English restaurant in the city centre.",
            "explanation": "The output sentence faithfully converts the data in the input into a natural-sounding sentence."
        },
        {
            "input": "name[The Rice Boat], food[Indian], priceRange[cheap], customer rating[5 out of 5], area[city centre], familyFriendly[yes], near[Express by Holiday Inn]",
            "output": "The Rice Boat is a cheap Indian restaurant in the center of the city near Express by Holiday Inn. It is family friendly and has the highest customer rating.",
            "explanation": "The output sentence correctly contains all the information from the input. It is grammatical and idiomatic."
        },
    ],
    "Negative Examples": [
        {
            "input": "name[Blue Spice], eatType[coffee shop], priceRange[more than £30], customer rating[5 out of 5], area[riverside], familyFriendly[yes], near[Avalon]",
            "output": "Blue Spice is a Colombian coffee shop located by the riverside, near Avalon in Boston. Its prices are over £30. Its customer ratings are 5 out of 5.",
            "explanation": "While the output contains most information from the input, it hallucinates by adding ungrounded information such as \"Colombian\" and \"Boston\"."
        },
        {
            "input": "name[The Waterman], food[Japanese], priceRange[less than £20], customer rating[average], area[riverside], familyFriendly[yes]",
            "output": "The Waterman is an expensive family-friendly Chinese restaurant with average customer rating in riverside",
            "explanation": "The output contains incorrect information such as \"expensive\" and \"Chinese restaurant\""
        }
    ],
    "Instances": Instances
}

# export
with open('task949_e2e_text_generation.json', 'w') as fp:
    final_json = json.dumps(task_json, indent=4, ensure_ascii=False)
    print(final_json, file=fp)